# An interpretable prediction system for the Site Of Origin (SOO) of Outflow Tract Ventricular Arrhythmias (OTVAs)

## Introduction

Outflow Tract Ventricular Arrhythmias (OTVAs) are premature ventricular beats. In structurally normal hearts, they often present characteristic ECG morphologies but can be challenging to manually localize precisely without invasive mapping.

We can consider each patient **case** to generally consist of:
- A 12-lead **ECG signal** segment (≈2.5 s)  
- Basic **demographic data** (age, sex, height,...)
- A (clinician-assigned) **label** indicating the arrhythmia's Site of Origin (SOO)

We aim to build a system of models that can *learn* from this data and predict the **Site Of Origin (SOO)** of OTVAs. The goal is to be able to generalize to new patient cases, and have both 1) interpretability of the learned criteria used for classification and 2) a fast inference time per prediction.

We will complete this broad task in two parts:

1. **Part 1: Left vs. Right Outflow Tract**  
   Classify each arrhythmia focus as coming from the Left Ventricular Outflow Tract (LVOT) or the Right Ventricular Outflow Tract (RVOT).

2. **Part 2: Sub-regional Localization**  
   On those beats classified as **LVOT** in Part 1, further discriminate between origins at the **Right Coronary Cusp (RCC)** and the **aortomitral commissure** (the mitral-aortic continuity), as assigned to our group D. This is in our case a sub-classification task, since the Right Coronary Cusp (RCC) is one of the three aortic cusps (alongside LCC and NCC) sitting in the left ventricular outflow tract; while the “commissure” in this context is the junction either between the RCC & LCC or the mitral-aortic continuity, being in both cases a region of the LVOT.